In [3]:
import h2o
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "1.8.0_171"; OpenJDK Runtime Environment (build 1.8.0_171-8u171-b11-0ubuntu0.18.04.1-b11); OpenJDK 64-Bit Server VM (build 25.171-b11, mixed mode)
  Starting server from /home/sajeewa/anaconda2/lib/python2.7/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpJifc_g
  JVM stdout: /tmp/tmpJifc_g/h2o_sajeewa_started_from_python.out
  JVM stderr: /tmp/tmpJifc_g/h2o_sajeewa_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321... successful.


H2O cluster uptime:,01 secs
H2O cluster timezone:,Asia/Colombo
H2O data parsing timezone:,UTC
H2O cluster version:,3.18.0.11
H2O cluster version age:,2 months and 2 days
H2O cluster name:,H2O_from_python_sajeewa_ykd1d6
H2O cluster total nodes:,1
H2O cluster free memory:,2.158 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"accepting new members, healthy"


In [12]:
from h2o.estimators.glm import H2OGeneralizedLinearEstimator as glm
from builtins import range

In [31]:
train = h2o.import_file('data/train_features_preprocessed.csv')
# train['donated'] = train['donated'].asfactor()

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [32]:
predictors = ["mon_since_last", "num_donations", "mon_since_first", "num_donations_per_month", "tenure_ratio"]
response_col = "donated"

In [33]:
glm_model = glm(family= "binomial", lambda_search=True, solver='COORDINATE_DESCENT')
glm_model.train(predictors, response_col, training_frame= train)
glm_regularization_path = glm.getGLMRegularizationPath(glm_model)

glm_model2 = glm.makeGLMModel(model=glm_model, coefs=glm_regularization_path['coefficients'][10])
dev1 = glm_regularization_path['explained_deviance_train'][10]
p = glm_model2.model_performance(train)
dev2 = 1-p.residual_deviance()/p.null_deviance()
print(dev1," =?= ",dev2)

assert abs(dev1 - dev2) < 1e-6

for l in range(0,len(glm_regularization_path['lambdas'])):
    glm_model = glm(family='binomial',lambda_search=False,Lambda=glm_regularization_path['lambdas'][l],solver='COORDINATE_DESCENT')
    glm_model.train(predictors, response_col, training_frame= train)
    cs = glm_regularization_path['coefficients'][l]
    cs_norm = glm_regularization_path['coefficients_std'][l]
    diff = 0
    diff2 = 0
    for n in cs.keys():
        diff = max(diff,abs((cs[n] - glm_model.coef()[n])))
        diff2 = max(diff2,abs((cs_norm[n] - glm_model.coef_norm()[n])))
    print(diff)
    print(diff2)
    assert diff < 1e-2
    assert diff2 < 1e-2
    p = glm_model.model_performance(train)
    devm = 1-p.residual_deviance()/p.null_deviance()
    devn = glm_regularization_path['explained_deviance_train'][l]
    print(devm)
    print(devn)
    assert abs(devm - devn) < 1e-4

glm Model Build progress: |███████████████████████████████████████████████| 100%
(0.08443529132068783, ' =?= ', 0.08443529132074679)
glm Model Build progress: |███████████████████████████████████████████████| 100%
0
0
0.0
0.0
glm Model Build progress: |███████████████████████████████████████████████| 100%
1.03761443881e-12
9.10965747281e-13
0.00695075914382
0.00695075914365
glm Model Build progress: |███████████████████████████████████████████████| 100%
8.12801120259e-05
1.27601000437e-05
0.0152130186707
0.0152169273414
glm Model Build progress: |███████████████████████████████████████████████| 100%
6.21253590616e-05
1.42720506143e-05
0.0263779141667
0.026379226293
glm Model Build progress: |███████████████████████████████████████████████| 100%
0.000204798209794
8.16489928168e-05
0.0378245733398
0.0378341938975
glm Model Build progress: |███████████████████████████████████████████████| 100%
0.000242474469544
0.000127080152543
0.048185390992
0.0481990827489
glm Model Build progress: |██

In [34]:
test = h2o.import_file('data/test_data_preprocessed.csv')

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [35]:
test['donated'] = None
# test['donated'] = test['donated'].asfactor()

In [51]:
test

mon_since_last,num_donations,mon_since_first,num_donations_per_month,tenure_ratio,donated
2,12,52,0.230769,0.0384615,nan
21,7,38,0.184211,0.552632,nan
4,1,4,0.25,1,nan
11,11,38,0.289474,0.289474,nan
4,12,34,0.352941,0.117647,nan
3,21,42,0.5,0.0714286,nan
4,2,4,0.5,1,nan
14,1,14,0.0714286,1,nan
23,2,87,0.0229885,0.264368,nan
14,4,64,0.0625,0.21875,nan


In [38]:
predictions = glm_model.predict(test)

glm prediction progress: |████████████████████████████████████████████████| 100%


In [52]:
y = h2o.import_file('data/test_data.csv')

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [56]:
ids=y['C1']

In [58]:
train

mon_since_last,num_donations,mon_since_first,donated,num_donations_per_month,tenure_ratio
2,50,98,1,0.510204,0.0204082
0,13,28,1,0.464286,0
1,16,35,1,0.457143,0.0285714
2,20,45,1,0.444444,0.0444444
1,24,77,0,0.311688,0.012987
4,4,4,0,1,1
2,7,14,1,0.5,0.142857
1,12,35,0,0.342857,0.0285714
5,46,98,1,0.469388,0.0510204
0,3,4,0,0.75,0


In [66]:
predictions_class1 = [ids, 'Made Donation in March 2007':predictions['p1']]

In [69]:
h2o.export_file(predictions['p1'], "submission_h2o_glm.csv", force=True)

Export File progress: |███████████████████████████████████████████████████| 100%


In [70]:
h2o.export_file(ids, "ids.csv", force=True)

Export File progress: |███████████████████████████████████████████████████| 100%
